In [23]:
import os  
import random


 Using the already preprocessed datasets (with labels). We do different actions still:
 
 - Out-of-time Train-Test split (don't necesserily delete overlap)
 - For different percentages of the cases (!), not activities, in the Training log change label. This change then has to be applied to all activities of this case (of course).
 
 

# Helper functions

In [24]:
from DatasetManager import DatasetManager

import pandas as pd

In [25]:
def flip_label(datadum, case_id_col, fraction):
    data = datadum.copy(deep=True)
    data.rename(columns={case_id_col: 'Case_ID'}, inplace=True)
    #get a list with all case_IDs of only those cases which have a positive /defiant label
    case_ids = list(set(data.loc[data['label'] != 'regular']['Case_ID'].tolist()))
    print(len(case_ids))
    #sample fraction of these IDs
    k = int(len(case_ids) * fraction)
    indices = random.sample(range(len(case_ids)), k)
    print(len(indices))
    case_ids_to_flip = [case_ids[i] for i in indices]
    #for each of the IDs we sampled change the label for each activity
    data.loc[data.Case_ID.isin(case_ids_to_flip), 'label'] = 'regular'
    data.rename(columns={'Case_ID': case_id_col}, inplace=True)
    return data

In [32]:
def change_name(logname):
    if logname ==  "production":
        with_caps = "Production"
    if logname ==  "bpic2015_1_f2":
        with_caps = "BPIC15_1_f2"
    if logname ==  "bpic2015_2_f2":
        with_caps = "BPIC15_2_f2"
    if logname ==  "bpic2015_3_f2":
        with_caps = "BPIC15_3_f2"
    if logname ==  "bpic2015_4_f2":
        with_caps = "BPIC15_4_f2"
    if logname ==  "bpic2015_5_f2":
        with_caps = "BPIC15_5_f2"
    if logname ==  "bpic2011_f1":
        with_caps = "BPIC11_f1"
    if logname ==  "bpic2011_f2":
        with_caps = "BPIC11_f2"
    if logname ==  "bpic2011_f3":
        with_caps = "BPIC11_f3"
    if logname ==  "bpic2011_f4":
        with_caps = "BPIC11_f4"
    if logname ==  "sepsis_cases_1":
        with_caps = "sepsis_cases_1"
    if logname ==  "sepsis_cases_2":
        with_caps = "sepsis_cases_2"
    if logname ==  "sepsis_cases_4":
        with_caps = "sepsis_cases_4"
    if logname ==  "bpic2017_cancelled":
        with_caps = "BPIC17_O_Cancelled" 
    if logname ==  "bpic2012_cancelled":
        with_caps = "bpic2012_O_CANCELLED-COMPLETE"    
    return with_caps

In [33]:
def get_ratio(datadum, case_id_col):
    data = datadum.copy(deep=True)
    data.rename(columns={case_id_col: 'Case_ID'}, inplace=True)
    case_ids_reg = list(set(data.loc[data['label'] == 'regular']['Case_ID'].tolist()))
    case_ids_dev = list(set(data.loc[data['label'] != 'regular']['Case_ID'].tolist()))
    print("Amount reg:", len(case_ids_reg))
    print("Amount dev:", len(case_ids_dev))

In [34]:
def create_PU_datasets(logname):
    #read data
    dataset_manager = DatasetManager(logname)
    newname = change_name(logname)
    data = pd.read_csv('Original/'+newname+'.csv', sep=';')
    
    if logname in ['bpic2011_f1', 'bpic2011_f2', 'bpic2011_f3', 'bpic2011_f4','bpic2015_1_f2','bpic2015_2_f2','bpic2015_3_f2','bpic2015_4_f2','bpic2015_5_f2','sepsis_cases_1','sepsis_cases_2','sepsis_cases_4']:
        data['time:timestamp'] = pd.to_datetime(data['time:timestamp'])
    if logname in ['bpic2012_accepted', 'bpic2012_cancelled', 'bpic2012_declined']:
        data['Complete Timestamp'] = pd.to_datetime(data['Complete Timestamp'])

    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols,
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}
    case_id_col = dataset_manager.case_id_col
    # split into training and test 
    #out_of_time_split: needs to split of all traces which have at least one activity after a certain point in time
    #we delete the overlap traces
    
    
    
    train, test = dataset_manager.split_data_strict(data, train_ratio=0.8, split="temporal")
    
    get_ratio(train, case_id_col)
    get_ratio(test, case_id_col)
    
    print(train['label'].value_counts())
    #save training and test log without flips 
    train.to_csv('Train_PU00_'+newname+'.csv', index=False, sep=';')
    test.to_csv('Test_'+newname+'.csv', index=False, sep=';')
    #get different percentages of traces in training log with positive label to flip
    #save every of these logs
    train_ori = train.copy(deep=True)
    train_25 = flip_label(train, case_id_col, 0.25)
    print(train_25['label'].value_counts())
    train_25.to_csv('Train_PU25_'+newname+'.csv', index=False, sep=';')
    train = train_ori.copy(deep=True)
    train_50 = flip_label(train, case_id_col, 0.5)
    print(train_50['label'].value_counts())
    train_50.to_csv('Train_PU50_'+newname+'.csv', index=False, sep=';')
    train = train_ori.copy(deep=True)
    train_75 = flip_label(train, case_id_col, 0.75)
    print(train_75['label'].value_counts())
    train_75.to_csv('Train_PU75_'+newname+'.csv', index=False, sep=';')
    
    


In [7]:
create_PU_datasets('bpic2017_cancelled')

Amount reg: 13192
Amount dev: 11938
Amount reg: 3320
Amount dev: 2963
regular    524417
deviant    403368
Name: label, dtype: int64
11938
2984
regular    625501
deviant    302284
Name: label, dtype: int64
11938
5969
regular    724771
deviant    203014
Name: label, dtype: int64
11938
8953
regular    826816
deviant    100969
Name: label, dtype: int64


In [8]:
create_PU_datasets('bpic2012_cancelled')

Amount reg: 2382
Amount dev: 1366
Amount reg: 663
Amount dev: 274
regular    93627
deviant    44380
Name: label, dtype: int64
1366
341
regular    104594
deviant     33413
Name: label, dtype: int64
1366
683
regular    115657
deviant     22350
Name: label, dtype: int64
1366
1024
regular    127083
deviant     10924
Name: label, dtype: int64


In [14]:
create_PU_datasets('sepsis_cases_1')

Amount reg: 532
Amount dev: 93
Amount reg: 139
Amount dev: 18
regular    8613
deviant    1769
Name: label, dtype: int64
93
23
regular    8966
deviant    1416
Name: label, dtype: int64
93
46
regular    9604
deviant     778
Name: label, dtype: int64
93
69
regular    9850
deviant     532
Name: label, dtype: int64


In [15]:
create_PU_datasets('sepsis_cases_2')

Amount reg: 540
Amount dev: 85
Amount reg: 135
Amount dev: 22
regular    7815
deviant     760
Name: label, dtype: int64
85
21
regular    8000
deviant     575
Name: label, dtype: int64
85
42
regular    8177
deviant     398
Name: label, dtype: int64
85
63
regular    8363
deviant     212
Name: label, dtype: int64


In [21]:
create_PU_datasets('sepsis_cases_4')

Amount reg: 87
Amount dev: 538
Amount reg: 24
Amount dev: 133
deviant    8072
regular    1786
Name: label, dtype: int64
538
134
deviant    5985
regular    3873
Name: label, dtype: int64
538
269
regular    5653
deviant    4205
Name: label, dtype: int64
538
403
regular    7901
deviant    1957
Name: label, dtype: int64


In [10]:
create_PU_datasets('production')

Amount reg: 73
Amount dev: 103
Amount reg: 30
Amount dev: 14
deviant    1250
regular     824
Name: label, dtype: int64
103
25
regular    1133
deviant     941
Name: label, dtype: int64
103
51
regular    1421
deviant     653
Name: label, dtype: int64
103
77
regular    1729
deviant     345
Name: label, dtype: int64


In [11]:
create_PU_datasets('bpic2015_4_f2')

Amount reg: 382
Amount dev: 78
Amount reg: 101
Amount dev: 15
regular    15167
deviant     3519
Name: label, dtype: int64
78
19
regular    16001
deviant     2685
Name: label, dtype: int64
78
39
regular    17018
deviant     1668
Name: label, dtype: int64
78
58
regular    17876
deviant      810
Name: label, dtype: int64


In [12]:
create_PU_datasets('bpic2011_f1')

Amount reg: 563
Amount dev: 349
Amount reg: 119
Amount dev: 109
regular    34857
deviant    10470
Name: label, dtype: int64
349
87
regular    37251
deviant     8076
Name: label, dtype: int64
349
174
regular    40110
deviant     5217
Name: label, dtype: int64
349
261
regular    42593
deviant     2734
Name: label, dtype: int64


In [12]:
create_PU_datasets('bpic2011_f2')

Amount reg: 170
Amount dev: 742
Amount reg: 77
Amount dev: 151
deviant    67974
regular    29511
Name: label, dtype: int64
742
185
regular    49177
deviant    48308
Name: label, dtype: int64
742
371
regular    62956
deviant    34529
Name: label, dtype: int64
742
556
regular    81222
deviant    16263
Name: label, dtype: int64


In [13]:
create_PU_datasets('bpic2011_f3')

Amount reg: 717
Amount dev: 179
Amount reg: 145
Amount dev: 80
regular    40008
deviant     5630
Name: label, dtype: int64
179
44
regular    41250
deviant     4388
Name: label, dtype: int64
179
89
regular    43321
deviant     2317
Name: label, dtype: int64
179
134
regular    44636
deviant     1002
Name: label, dtype: int64


In [14]:
create_PU_datasets('bpic2015_5_f2')

C:\Users\u0132580\AppData\Local\Temp/ipykernel_20292/2295866648.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Original/'+newname+'.csv', sep=';')


Amount reg: 568
Amount dev: 272
Amount reg: 156
Amount dev: 55
regular    26616
deviant    15024
Name: label, dtype: int64
272
68
regular    30425
deviant    11215
Name: label, dtype: int64
272
136
regular    33947
deviant     7693
Name: label, dtype: int64
272
204
regular    38150
deviant     3490
Name: label, dtype: int64


In [29]:
create_PU_datasets('bpic2015_1_f2')

C:\Users\u0132580\AppData\Local\Temp/ipykernel_572/2295866648.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Original/'+newname+'.csv', sep=';')


Amount reg: 431
Amount dev: 124
Amount reg: 104
Amount dev: 36
regular    15377
deviant     5738
Name: label, dtype: int64
124
31
regular    16809
deviant     4306
Name: label, dtype: int64
124
62
regular    18245
deviant     2870
Name: label, dtype: int64
124
93
regular    19661
deviant     1454
Name: label, dtype: int64


In [30]:
create_PU_datasets('bpic2015_2_f2')

Amount reg: 482
Amount dev: 120
Amount reg: 125
Amount dev: 26
regular    23822
deviant     7186
Name: label, dtype: int64
120
30
regular    25474
deviant     5534
Name: label, dtype: int64
120
60
regular    27476
deviant     3532
Name: label, dtype: int64
120
90
regular    29121
deviant     1887
Name: label, dtype: int64


In [31]:
create_PU_datasets('bpic2015_3_f2')

C:\Users\u0132580\AppData\Local\Temp/ipykernel_572/2295866648.py:5: DtypeWarning: Columns (0,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Original/'+newname+'.csv', sep=';')


Amount reg: 868
Amount dev: 194
Amount reg: 199
Amount dev: 67
regular    34215
deviant     9544
Name: label, dtype: int64
194
48
regular    36486
deviant     7273
Name: label, dtype: int64
194
97
regular    38958
deviant     4801
Name: label, dtype: int64
194
145
regular    41173
deviant     2586
Name: label, dtype: int64


In [35]:
create_PU_datasets('bpic2011_f4')

Amount reg: 682
Amount dev: 230
Amount reg: 139
Amount dev: 89
regular    49273
deviant    15665
Name: label, dtype: int64
230
57
regular    53320
deviant    11618
Name: label, dtype: int64
230
115
regular    57785
deviant     7153
Name: label, dtype: int64
230
172
regular    61365
deviant     3573
Name: label, dtype: int64
